In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("prometheus-eval/prometheus-7b-v2.0")
model = AutoModelForCausalLM.from_pretrained("prometheus-eval/prometheus-7b-v2.0")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
conv = get_conv_template("mistral")

In [7]:
chat = [
  {"role": "system", "content": "You are an helpful assistant"},
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
]

In [8]:
prompt = tokenizer.apply_chat_template(chat, tokenize=False)

In [9]:
print(prompt)

<s> [INST] You are an helpful assistant

Hello, how are you? [/INST] I'm doing great. How can I help you today?</s>


In [1]:
from transformers import pipeline

In [2]:
model_name = 'PatronusAI/glider'
pipe = pipeline(
    "text-generation",
    model=model_name,
    max_new_tokens=1000,
    device="cuda",
    return_full_text=False
)

model-00001-of-00004.safetensors:  27%|##7       | 1.35G/4.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [3]:


messages = [
    {"role": "system", "content": "You answer questions if they are related to evaluating. Otherwise, you politely ask to talk about something else."}
    {"role": "user", "content": "Who am I?"},
]

result = pipe(messages)
print(result[0]['generated_text'])

config.json:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [14]:
from datasets import load_dataset

In [15]:
def load_and_preprocess_dataset(dataset_name):
    dataset = load_dataset(dataset_name)
    return dataset["train"]

In [ ]:
!pip install datasets

In [16]:
image_data = load_and_preprocess_dataset("Mausul/syn_dataset_no_evolution_multi_run_smol_v1")

Generating train split:   0%|          | 0/8828 [00:00<?, ? examples/s]

In [22]:
image_data[167]

{'original_question_id': '23ibLk5tF4wVKd7MMms9Yp',
 'question': 'What is the likelihood of the person with the umbrella tripping over the puddle in front of them?',
 'answer': 'Low',
 'rationales': ['The person with the umbrella is walking carefully, as evidenced by the umbrella being open and held above their head. This suggests that they are being cautious and are less likely to trip over the puddle.'],
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427>,
 'synthetic_question_id': '7NDTmN2W9Kd6bQAqeot8cA41w'}

In [3]:
import argparse
import torch
import os
import json
from tqdm import tqdm

In [2]:
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

Please install pyav to use video processing functions.


In [4]:
from PIL import Image
import math

In [5]:
model_path = 'kaist-ai/prometheus-vision-13b-v1.0'
model_name = 'llava-v1.5'
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, None, model_name)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loaded LLaVA model: kaist-ai/prometheus-vision-13b-v1.0


pytorch_model-00001-of-00003.bin:   2%|1         | 168M/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/zeus/miniconda3/

Model Class: LlavaLlamaForCausalLM


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [23]:
conv = conv_templates["llava_v1"].copy()
conv.append_message(conv.roles[0], "What is the likelihood of the person with the umbrella tripping over the puddle in front of them?")
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

In [8]:
prompt

"A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: How are you? ASSISTANT:"

In [9]:
conv.roles

('USER', 'ASSISTANT')

In [24]:
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

In [25]:
image = image_data[167]["image"]
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]

In [27]:
image_tensor.shape

torch.Size([3, 336, 336])

In [28]:
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

In [29]:
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor.unsqueeze(0).half().cuda(),
        do_sample=True,
        temperature=0.3,
        num_beams=1,
        max_new_tokens=500,
        use_cache=True
    )

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [30]:
input_token_len = input_ids.shape[1]
n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
if n_diff_input_output > 0:
    print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')

[Warning] 65 output_ids are not the same as the input_ids


In [37]:
outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
outputs = outputs.strip()
if outputs.endswith(stop_str):
    outputs = outputs[:-len(stop_str)]
outputs = outputs.strip()

In [36]:
outputs

'The likelihood of the person with the umbrella tripping over the puddle in front of them is relatively low. The person is likely to be aware of the puddle and take steps to avoid it, such as stepping over it or walking around it. Additionally, the umbrella could provide some protection against slipping on the wet surface. However, if the person is not paying attention or is distracted, there is a possibility that they could trip over the puddle. So the overall score is 3.'

In [3]:
def eval_qar(evaluator, question, image_description):
    eval_prompt = """You will be given an image and a question related to the image.

        Evaluate the quality of the question based on the following **criteria** and assign scores for each criterion on a scale of 1-5:
        1. Commonsense knowledge about human social behavior.
        2. Knowledge of the physical world.
        3. Visual understanding.
        4. Reasoning capabilities.
        5. Complexity requiring in-depth reasoning.

        ### Instructions:
        1. Use the provided image to assess the question and assign scores.
        2. For each criterion, assign a score (integer value) between 1 and 5, based on how well the question satisfies the criterion.
        3. Briefly justify the individual criterion scores in one or two sentences.
        4. Identify criteria where the question scored low or failed, with a short explanation.
        5. Suggest a method to evolve (improve) the question to better meet the criteria. Ensure that evolving the question to improve certain aspects does **not compromise or lower the quality of other aspects**.

        ### Important Note:
        - DO NOT hallucinate or infer details that cannot be directly observed or logically deduced from the description.
        - If the image is vague or incomplete, factor that into the evaluation process.

        ### Output Format:
        Provide the evaluation in the following structured format. Do NOT use strict JSON, but keep the structure clear and detectable:

        - **Scores**:
          - Commonsense: <score>
          - Physical World: <score>
          - Visual Understanding: <score>
          - Reasoning: <score>
          - Complexity: <score>
        - **Justification**: <brief justification of the individual scores>
        - **Failures**: <brief explanation of criteria not fully met>
        - **Evolution Method**: <method to evolve (improve) the question without lowering the quality of other aspects>

        Neither favor nor punish due to your internal bias. DO FAIR JUDGMENT."""

    print(eval_prompt)
    
    """
    query = f'Question (to be judged): {question}'

    messages = [
        {"role": "system", "content": eval_prompt},
        {"role": "user", "content": query},
    ]

    result = evaluator(messages)
    raw_eval_text = result[0]['generated_text']
    cleaned_eval_text = postprocess_judgement_details(slm, raw_eval_text)
    # out of 100
    score = aggregate_and_scale_score(cleaned_eval_text["scores"])
    evol_method = cleaned_eval_text["evolution_method"]
    return [score, evol_method]
    """

In [4]:
eval_qar(1,2,3)

You will be given an image and a question related to the image.

        Evaluate the quality of the question based on the following **criteria** and assign scores for each criterion on a scale of 1-5:
        1. Commonsense knowledge about human social behavior.
        2. Knowledge of the physical world.
        3. Visual understanding.
        4. Reasoning capabilities.
        5. Complexity requiring in-depth reasoning.

        ### Instructions:
        1. Use the provided image to assess the question and assign scores.
        2. For each criterion, assign a score (integer value) between 1 and 5, based on how well the question satisfies the criterion.
        3. Briefly justify the individual criterion scores in one or two sentences.
        4. Identify criteria where the question scored low or failed, with a short explanation.
        5. Suggest a method to evolve (improve) the question to better meet the criteria. Ensure that evolving the question to improve certain aspects doe